# DEMO#3 - Re-run with randint and unique indices per random subset...

In [1]:
!python --version

Python 3.10.6


In [2]:
import shap

AttributeError: module 'llvmlite.binding' has no attribute 'get_host_cpu_name'

In [3]:
!env | grep PATH

SLURM_EXPORT_ENV=PATH,LANG,JUPYTERHUB_API_TOKEN,JPY_API_TOKEN,JUPYTERHUB_ADMIN_ACCESS,JUPYTERHUB_CLIENT_ID,JUPYTERHUB_HOST,JUPYTERHUB_OAUTH_CALLBACK_URL,JUPYTERHUB_USER,JUPYTERHUB_SERVER_NAME,JUPYTERHUB_API_URL,JUPYTERHUB_ACTIVITY_URL,JUPYTERHUB_BASE_URL,JUPYTERHUB_SERVICE_PREFIX,USER,HOME,SHELL
PATH=/app/jupyter/ilab/lab-env/bin:/panfs/ccds02/app/modules/anaconda/platform/x86_64/rhel/8.6/3-2022.05/condabin:/usr/local/bin:/usr/bin:/bin:/usr/cumulus/bin:/sbin:/usr/sbin


In [ ]:
conda list | grep shap

In [ ]:
pip freeze | grep shap

# DEMO#2 - Expand Experiments...

# DEMO#1 - 01/31/2024 - See https://jh-ml.nccs.nasa.gov/jupyterhub-prism/user/gtamkin/lab/workspaces/auto-0/tree/_AGB-dev/mpf-model-factories/MultiPathFusion/multi_path_fusion/notebooks/glenn/gt_mc_get_stats.ipynb

# Generate Permutation Importance (i.e., rank) from SHAP files

In [ ]:
#import shap
import numpy as np
import pickle
import glob
import pandas as pdconda
import subprocess
import csv
from pathlib import Path

In [ ]:
def format_band_set(subset):
    indexListIntStr = ''
    count = 0
    for band in subset:
        if (count == 0):
            indexListIntStr = indexListIntStr + str(band)
        else:
            indexListIntStr = indexListIntStr + ', ' + str(band)
        count = count + 1
    return indexListIntStr


# Read from CSV

In [ ]:
hyperspectral_aggregate_bandnames = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate-bandnames.csv'
only_row = None
with open(hyperspectral_aggregate_bandnames) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        only_row = row
        print(row[0])
        print(row[446])
#print(only_row)

In [ ]:
# initialize list of lists
headers = ['Group','Top10Index','Top10Rank','Top10Value','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
site = 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0'

## Read batch of shap files for experiment, which contains run-time artifacts (e.g., SHAP values)

In [ ]:
root_data_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output'

In [ ]:
epochs_results_ALL_nocallbacks = [
     ['ALL-epochs-30','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','477', '1016', '10', '10','0:12:26.46','ilab213','30','Pool', 
       'ALL_randintU_epochs30_20240502_1016','931628725990482715', '## ALL - 30 Epochs - ilab213 - started 05/02/2024: Trials = 10, Batch = 30, Goal = 1016',
       'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','NoEarlyWarning'],
]

In [ ]:
epochs_results_VI_nocallbacks = [
     ['V1-epochs-30','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:07:04.33','gpu005','10','Pool', 
       'VI/combinations_sets_20240430_gpu005_collection-VI_epochs-30_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','633569878306525734', '## VI - 30 Epochs - gpu005 - started 04/30/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     ['V1-epochs-30','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:35:44','ilab213','10','Pool', 
       'VI/combinations_sets_20240430_ilab213_collection-VI_epochs-30_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','392917562366869977', '## VI - 30 Epochs - ilab213 - started 04/30/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     ['V1-epochs-30','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:56:34','ilab209','10','Pool', 
       'VI/combinations_sets_20240501_ilab209_collection-VI_epochs-30_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','770164563465191834', '## VI - 30 Epochs - ilab209 - started 05/01/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     ['V1-epochs-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:12:26.46','gpu005','10','Pool', 
       'VI/combinations_sets_20240501_gpu005_collection-VI_epochs-50_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','854448484021496104', '## VI - 50 Epochs - gpu005 - started 05/01/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     ['V1-epochs-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:18:22.76','ilab213','10','Pool', 
       'VI/combinations_sets_20240501_ilab213_collection-VI_epochs-50_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','508386558032476138', '## VI - 50 Epochs - ilab213 - started 05/01/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     ['V1-epochs-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:56:34.53','ilab209','10','Pool', 
       'VI/combinations_sets_20240430_ilab209_collection-VI_epochs-50_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','799261882294034652', '## VI - 50 Epochs - ilab209 - started 04/30/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     ['V1-epochs-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:28:04.49','ilab207','10','Pool', 
       'VI/combinations_sets_20240430_ilab207_collection-VI_epochs-50_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','647254132847848174', '## VI - 50 Epochs - ilab207 - started 04/30/2024: Trials = 10, Batch = 10, Goal = 32',
       'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
     # ['V1-epochs-30','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '21','0:17:30.73','ilab207','10','Pool', 
     #   'VI/combinations_sets_20240501_ilab207_collection-VI_epochs-30_min_occurrences-10_batch_size-10_num_samples-32_bandLen-21','502742172878765632', '## VI - 30 Epochs - ilab207 - started 05/01/2024: Trials = 10, Batch = 10, Goal = 32',
     #   'MLBS_2018_hyperspectral_indices.tif','NoEarlyWarning'],
]

In [ ]:
permutation_results_VI_pairwise = [
     ['V1-pairs-shap-1000','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','combinations','21', '210', '2', '10','??:27:00.00','gpu016','40','Pool', 
       'VI/combinations_sets_20240405_sample_size-1000_collection-VI_batch_size-2_num_samples-210_bandLen-21','247881137164239533', '## VI - sample_size-1000',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['V1-pairs-shap-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','combinations','21', '210', '2', '10','??:27:00.00','ilab213','30','Pool', 
       'VI/combinations_sets_20240326_collection-VI_batch_size-2_num_samples-210_bandLen-21','874262913239269001', '## VI - 1 pair of bands per trial until all are paired once',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['V1-pairs-shap-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','combinations','21', '210', '2', '10','??:27:00.00','ilab213','30','Pool', 
       'VI/combinations_sets_20240401_collection-VI_batch_size-2_num_samples-210_bandLen-21','686296879186232154', '## VI - 1 pair of bands per trial until all are paired once',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['V1-pairs-shap-50','NoTop10Index', 'Top10Rank', 'Top10Value','10Epochs','combinations','21', '210', '2', '10','??:27:00.00','ilab213','30','Pool', 
       'VI/combinations_sets_20240402_collection-VI_batch_size-2_num_samples-210_bandLen-21_epochs-10','110930597116663652', '## VI - 10 epochs - 1 pair of bands per trial until all are paired once',
       'MLBS_2018_hyperspectral_indices.tif','10 Epochs'],
     ['V1-pairs-shap-50','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','permutations','21', '420', '2', '10','??:27:00.00','ilab213','30','Pool', 
       'VI/permutations_sets_20240401_collection-VI_batch_size-2_num_samples-420_bandLen-21','721589469845150807', '## VI - 1 pair of bands per trial until all are paired once',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
]

In [ ]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_VI_exp1 = [
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '188', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_188_20m_3b','575050868397492482', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '191', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_191_20m_3b','972812867574749608', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '193', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_193_20m_3b','853379430912402815', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '200', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_200_20m_3b','875083503082867641', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '214', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_214_20m_3b','279550132438151710', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '225', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_225_20m_3b','424433684070503325', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '230', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_230_20m_3b','500424548463509068', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '247', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_247_20m_3b','498345501911623626', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
]
#    
      # ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '188', '3', '20','~0:27:00.00','gpu007','40','Pool', 
      #  'VI/VI_randintU_20240302_188_20m_3b','575050868397492482', '## VI - 20 minimum occurrences, 3 bands per trial',
      #  'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
permutation_results_VI_exp2 = [
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '191', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_191_20m_3b','806398069851382119', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '193', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_193_20m_3b','874529206487521041', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '200', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_200_20m_3b','965060104529880310', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '214', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_214_20m_3b','728278796873995749', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '225', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_225_20m_3b','514050032010718833', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '230', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_230_20m_3b','360349938924147892', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '247', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_247_20m_3b','214107507827042739', 'VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
]
#    
      # ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '188', '3', '20','~0:27:00.00','gpu002','40','Pool', 
      #  'VI/VI_randintU_20240303_188_20m_3b','898188862101410804', '## VI - 20 minimum occurrences, 3 bands per trial',
      #  'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
permutation_results_VI_exp3 = [
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '191', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_191_20m_3b','101554869495136186', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '193', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_193_20m_3b','261914886327368313', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '200', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_200_20m_3b','552713534466588559', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '214', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_214_20m_3b','121679581342470880', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '225', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_225_20m_3b','664133247155684072', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '230', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_230_20m_3b','131603933973007654', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '247', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240303_247_20m_3b','120201914459639429', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
]
permutation_results_VI_exp_summary = permutation_results_VI_exp1 + permutation_results_VI_exp2 + permutation_results_VI_exp3


In [ ]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_VI = [
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:27:25.246467','ilab213','30','Pool', 
       'VI_randint_U_20240229_6','891516017211874507', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:27:25.246467',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:26:36.387707','ilab213','30','Pool', 
      'VI_randint_U_20240229_5','345536919353575236', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:36.387707',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:25:12.317','ilab213','30','Pool', 
      'VI_randint_U_20240229_4','673843053471350852', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:25:12.317',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:26:52.317282','ilab213','30','Pool', 
      'VI_randint_U_20240229_3','548764528868727045', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:52.317282',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     # ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:26:56.519002','ilab213','30','Pool', 
     #  'VI_randint_U_20240229_2','418094143373180009', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:56.519002',
     #  'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:27:43.264823','ilab213','30','Pool', 
      'VI_randint_U_20240229_1','711241757083075094', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:27:43.264823',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
]

In [ ]:
#headers = ['Group','Top10Index','Top10Rank','Top10Value','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_ALL_randint_U = [
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '1016', '10', '10','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240229_1016','983932495984268956', 
      'ALL - gpu002 - 10min - GOAL=1016 ~?? hours',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '1032', '10', '10','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240305_1032','763030221589651938', 
      'ALL - gpu002 - started 03/05/2024:  Min 10, Trials = 10, Batch = 40, Goal = 1032',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '1653', '10', '20','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240305_1653','882848942719320298', 
      'ALL - gpu002 - started 03/05/2024:  Min 20, Trials = 10, Batch = 40, Goal = 1653',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '3108', '10', '50','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240301_3108','195018083738905409', 
      'ALL - gpu002 - 50min - GOAL=3108',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '6064', '10', '100','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240301_6064','215041145914163553', 
      'ALL - ilab213 - 100m- GOAL=6064',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
]


In [ ]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_NEON = [
     ['_NEON','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randit','426', '5666', '10', '100', '07:00.00','gpu004','40','Pool', 
      'randint_sets_20240215_collection-NEON_max_occurrences-100_batch_size-10_num_samples-5666_bandLen-426','188577871478003488', 
      'NEON on gpu004 - min 10 occurrences: ~7hours',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
]


In [ ]:
permutation_results_VI_exp1_top3 = permutation_results_VI_exp1[:3]
permutation_results_VI_top3 = permutation_results_VI[:3]

#permutation_results_VI_exp1_top3
#permutation_results_VI_top3

In [ ]:
#permutation_results = permutation_results_ALL
#permutation_results = permutation_results_VI
#permutation_results = permutation_results_NEON
#permutation_results = permutation_results_ALL + permutation_results_NEON + permutation_results_VI
#permutation_results = permutation_results_ALL + permutation_results_NEON + permutation_results_VI
#permutation_results = permutation_results_VI_3trials
#permutation_results = permutation_results_ALL_randint_U + permutation_results_ALL  + permutation_results_NEON + permutation_results_VI_3trials + permutation_results_VI
#permutation_results = permutation_results_ALL_randint_U  + permutation_results_NEON + permutation_results_VI_pairwise + permutation_results_VI_3trials + permutation_results_VI
#permutation_results = permutation_results_VI_pairwise
#permutation_results = epochs_results_ALL_nocallbacks + epochs_results_VI_nocallbacks + permutation_results_ALL_randint_U + permutation_results_VI_pairwise + permutation_results_VI_exp1 + permutation_results_VI + permutation_results_NEON 
permutation_results = permutation_results_ALL_randint_U + epochs_results_ALL_nocallbacks + permutation_results_VI_pairwise + permutation_results_VI_exp1_top3 + permutation_results_VI_top3 + epochs_results_VI_nocallbacks + permutation_results_NEON 
#permutation_results = epochs_results_ALL_nocallbacks + epochs_results_VI_nocallbacks + permutation_results_ALL_randint_U + permutation_results_VI_pairwise + permutation_results_VI_exp_summary + permutation_results_VI + permutation_results_NEON 
#permutation_results = permutation_results_ALL_randint_U + permutation_results_VI_pairwise + permutation_results_VI_3trials + permutation_results_VI + permutation_results_NEON 
#permutation_results = permutation_results_ALL_randint_U + permutation_results_VI_pairwise + permutation_results_VI_pairwise_big_batch + permutation_results_VI_3trials + permutation_results_VI + permutation_results_NEON 
print(len(permutation_results), '\n',permutation_results)

In [ ]:
# Create the pandas DataFrame
import pandas as pd
df = pd.DataFrame(permutation_results, columns=headers)
#df

# Loop through result files and create the ranking

In [ ]:
#. Loop through result files and create the ranking
for r_index, row in df.iterrows():
    rank_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/' + df.at[r_index,'ResultDir'] + '.rank' 
#    rank_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/VI-20min-3trials/' + df.at[r_index,'ResultDir'] + '.rank' 
    my_file = Path(rank_file)
    if my_file.is_file():
        bandAbsSumArr = pickle.load(open(rank_file, "rb"))
        print('\n ======== index[', r_index, '] ==========')
        print('Loaded:', rank_file)
    else:
        shap_dir_prefix = root_data_path + '/' + df.at[r_index,'ResultDir'] + '/' + df.at[r_index,'ID'] 
        shap_dir = shap_dir_prefix + '/PERMUTATION_IMPORTANCE_VALUES/'
        print('\n ======== index[', r_index, '] ==========')
        print(r_index,'index: ', shap_dir)

        # Returns a list of names in list files. 
        shap_files = glob.glob(shap_dir + '**/*.shap_values0to50',  
                           recursive = True) 

        # set to max number of hyperspectral indices in experiment
        bandLen = int(df.at[r_index,'#Bands'])
        bandOccurenceArr = np.zeros(bandLen).astype(int)
        bandAbsSumArr = np.zeros(bandLen)

        first_time = True
        index = 0
        # Reload config
        start = left = '[['
        end = right = ']]'

        for shap_vals_path in shap_files:

            index = index+1
            num_shap_files = len(shap_files)
            bandListStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
            bandList = [e for e in bandListStr.split(',')]
            for bandNum in bandList:
                bandOccurenceArr[int(bandNum)-1] = (bandOccurenceArr[int(bandNum)-1]) + 1

            # Reload shap values
            shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
            num_bins = len(shap_values0to50)
            num_rows = len(shap_values0to50[0])
            num_bands = num_cols = len(shap_values0to50[0][0])

            #  Walk across bins [0,4]
            for _bin in range(0, num_bins):

                # get absolute value of all values in matrix
                shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])

                # convert numpy matrix to dataframe and print sum of absolute values in each column
                df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)

                # create a row (numpy array) where each column is the sum of the absolute values in that vertical column
                shap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)

                # sort the row from low to high to determine the rank of the columns (e.g., features)
                shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)

                # loop thru each column and update the corresponding statistics in the global row
                for _col in range(0, num_cols):
                    bandNum = int(bandList[_col])

                    # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
                    # this allows us to keep a running total shap values per feature for downselection later....
                    bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + df_shap_values0to50_bin_abs[_col].sum())

#        print('Summary array before sort:', bandAbsSumArr)
        ascending_indices = np.argsort(bandAbsSumArr)[:10]
        print('Top ten before after:', ascending_indices)
        minIndex = ascending_indices[0]

        descending_indices = np.argsort(bandAbsSumArr)[::-1][:10]
        maxIndex = descending_indices[0]

        pickle.dump(bandAbsSumArr, open(rank_file , "wb"))
        print('Saved:', rank_file)
        
    with np.printoptions(precision=5, suppress=True):
#        print('Final:', bandAbsSumArr[:10])
        print('Final.min:', bandAbsSumArr.min(), 'Final.max:', bandAbsSumArr.max())
        print('Final L->H:', np.argsort(bandAbsSumArr)[:10])
        row['Top10Index'] = np.argsort(bandAbsSumArr)[::-1][:10]

        indexList = np.argsort(bandAbsSumArr)[::-1][:10]
        bandList = []
        valueList = []
        for index in range(0, 10):
            bandList.append(str(only_row[indexList[index]]))
            valueList.append(str("{:3.3f}".format(bandAbsSumArr[indexList[index]])))
            print(indexList[index], only_row[indexList[index]],"{:3.3f}".format(bandAbsSumArr[indexList[index]]))        
            index = index + 1
        row['Top10Rank'] = bandList
        row['Top10Value'] = valueList

        print('Final Top10Rank Names:', bandList )
        print('Final Top10Rank Indices:', row['Top10Index'] )
        print('Final Top10Rank Cummulative Absolute SHAP values:', valueList)
        print('Final max:', row['Top10Index'][0])

In [ ]:
dfTest = df
dfTest

In [ ]:
print('[ID]-[Group]-[Min]-[Band]-[Trials]-[Scaling]-[Top 10 Rank]-[Top 10 Indices]-[Randomizer]')
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
for index, row in df.iterrows():
    print('', str(index).rjust(2), '-',row['Group'].rjust(4),'-', row['MinBands'].rjust(3), '-', row['#Bands'].rjust(3),'-', row['#Trials'].rjust(4), '-', row['Scaling'].rjust(8), '-', row['Top10Rank'], '-', row['Top10Index'], '-', row['Randomizer'])

In [ ]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.colheader_justify', 'left')
#dfSummary = df[['Group','MinBands','Band/Trial','#Trials','#Bands','Randomizer','Scaling','Top10Rank','Top10Value']] 
dfSummary = df[['Group','MinBands','Band/Trial','#Trials','#Bands','Randomizer','Top10Rank','Top10Value']] 
dfSummary.style.set_properties(**{'text-align': 'left'})
dfSummary

In [ ]:
from pandas import DataFrame
def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df


In [ ]:
left_align(dfSummary)

## +++++++++

In [ ]:
import pandas as pd
spreadsheet1 = pd.read_csv('/explore/nobackup/people/gtamkin/dev/srlite/test/srlite-2.0-rma-baseline/20240320/csv/WV02_20150911_M1BS_1030010049148A00_rmaSRLite_statistics.csv')

In [ ]:
spreadsheet1

In [ ]:
import pandas as pd
spreadsheet3 = pd.read_csv('/explore/nobackup/people/gtamkin/dev/srlite/test/srlite-2.0-rma-baseline/20240320_average_sampling/csv/WV02_20150911_M1BS_1030010049148A00_rmaSRLite_statistics.csv')

In [ ]:
spreadsheet3

In [ ]:
import pandas as pd
spreadsheet2 = pd.read_csv('/explore/nobackup/people/gtamkin/dev/srlite/test/srlite-2.0-rma-baseline/20240320-srlite_1.1.0/csv/WV02_20150911_M1BS_1030010049148A00_rma_SRLite_statistics.csv')

In [ ]:
spreadsheet2

In [ ]:
#random_set_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240229_collection-ALL_min_occurrences-100_batch_size-10_num_samples-6064_bandLen-447.pkl'
#random_set_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240229_collection-ALL_min_occurrences-50_batch_size-10_num_samples-3108_bandLen-447.pkl'
random_set_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240301_collection-VI_min_occurrences-20_batch_size-3_num_samples-188_bandLen-21.pkl'
random_set_rows = pickle.load(open(random_set_path, "rb"))
num_bins = len(random_set_rows)
num_rows = len(random_set_rows[0])
num_bands = num_cols = len(random_set_rows[0][0])
print (random_set_path, num_bins, num_rows, num_bands,num_cols, '\n', random_set_rows[0])

In [ ]:
df_random_set_rows = pd.DataFrame(data=random_set_rows)
df_random_set_rows.head()
print(list(df_random_set_rows.columns))
df_random_set_rows_sorted0 = df_random_set_rows.sort_values(0, ascending=True)
df_random_set_rows_sorted1 = df_random_set_rows_sorted0.sort_values(1, ascending=True)
df_random_set_rows_sorted2 = df_random_set_rows_sorted1.sort_values(2, ascending=True)
df_random_set_rows_sorted123 = df_random_set_rows.sort_values(by = [0, 1,2 ], ascending = [True, True, True])

df_random_set_rows_sorted = df_random_set_rows_sorted123
#sorted_df = df_random_set_rows.sort_values(by = [0, 1], ascending = [True, True])
#print(sorted_df)
with pd.option_context('display.max_rows', None,):
#     print(sorted_df)
     print(df_random_set_rows_sorted.head())

In [ ]:
df_random_set_rows_sorted0

In [ ]:
# Find Duplicate rows
with pd.option_context('display.max_rows', None,):
    #print(df_random_set_rows.groupby(df_random_set_rows.columns.tolist(),as_index=False).size())
    print(df_random_set_rows.value_counts().head())

In [ ]:
import ast
bandNamePairList = list(ast.literal_eval("[['blue_ccdc', 'BAND-B']]"))
bandNamePairList

In [ ]:
shap_vals_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/VI_randintU_20240302_188_20m_3b/575050868397492482/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE_50_epochs_callbacks::575050868397492482.keras[[1, 2, 4]].shap_values0to50'
shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
num_bins = len(shap_values0to50)
num_rows = len(shap_values0to50[0])
num_bands = num_cols = len(shap_values0to50[0][0])
print (shap_vals_path, num_bins, num_rows, num_bands,num_cols, '\n', shap_values0to50[0])

In [ ]:
shap_values0to50_bin = shap_values0to50[_bin]
df_shap_values0to50_bin = pd.DataFrame(data=shap_values0to50_bin)
df_shap_values0to50_bin

In [ ]:
shap_values0to50_bin = shap_values0to50[_bin]
df_shap_values0to50_bin = pd.DataFrame(data=shap_values0to50_bin)
df_shap_values0to50_bin

In [ ]:
shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])
df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)
df_shap_values0to50_bin_abs

In [ ]:
hap_values_sum_axis0 = np.sum(df_shap_values0to50_bin, axis=0)
hap_values_sum_axis0

In [ ]:
hap_values_sum_axis0_axis0 = np.sum(hap_values_sum_axis0, axis=0)
hap_values_sum_axis0_axis0

In [ ]:
hap_values_sum_axis0_axis1 = np.sum(hap_values_sum_axis0, axis=1)
hap_values_sum_axis0_axis1

In [ ]:
hap_values_sum = np.sum(df_shap_values0to50_bin)
hap_values_sum

In [ ]:
hap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)
hap_values_abs_sum_axis0

In [ ]:
hap_values_abs_sum = np.sum(df_shap_values0to50_bin_abs[0])
hap_values_abs_sum

In [ ]:
dfSort = dfTest.sort_values(["Group","ID"])
dfSort